In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews
    !pip install -U ipykernel
    !pip install joypy
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Complementary_Files/Datos.csv')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Results')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Figures')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Result')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on colab
/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project


In [2]:
# from __future__ import print_function
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from six.moves import xrange
import datetime
import gc
import pandas as pd
import joypy

from scipy import signal
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform


#from ResidualStack import ResidualStack
#from Residual import Residual

from Jaguas_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()

from datetime import timedelta
import wandb
from wandb import AlertLevel

from sklearn.manifold import TSNE

import pickle as pkl

In [3]:
dir = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Jaguas_2018'

In [4]:
"""Cuenta la cantidad de archivos en cada carpeta de un directorio"""
total_archivos = 0
for raiz, carpetas, archivos in os.walk(dir):
  carpetas.sort()
  # Obtener la cantidad de archivos en la carpeta actual
  cantidad_archivos = len(archivos)  
  # Sumar la cantidad de archivos actual a la variable total_archivos
  total_archivos += cantidad_archivos      
  # Imprimir el resultado
  print(f"Grabadora: {os.path.basename(raiz)} Cantidad Grabaciones: {cantidad_archivos}")
print(f"Total grabaciones de todas las grabadoras: {total_archivos}")

Grabadora: Jaguas_2018 Cantidad Grabaciones: 0
Grabadora: G03_m Cantidad Grabaciones: 565
Grabadora: G04_m Cantidad Grabaciones: 418
Grabadora: G06_m Cantidad Grabaciones: 887
Grabadora: G07_m Cantidad Grabaciones: 621
Grabadora: G08_m Cantidad Grabaciones: 648
Grabadora: G09_m Cantidad Grabaciones: 908
Grabadora: G13_m Cantidad Grabaciones: 636
Grabadora: G15_m Cantidad Grabaciones: 874
Grabadora: G17_m Cantidad Grabaciones: 870
Grabadora: G19_m Cantidad Grabaciones: 886
Grabadora: G23_m Cantidad Grabaciones: 555
Grabadora: G24_m Cantidad Grabaciones: 554
Grabadora: G25_m Cantidad Grabaciones: 645
Grabadora: G27_m Cantidad Grabaciones: 644
Grabadora: G28_m Cantidad Grabaciones: 553
Grabadora: G29_m Cantidad Grabaciones: 553
Grabadora: G34_m Cantidad Grabaciones: 664
Grabadora: G35_m Cantidad Grabaciones: 627
Grabadora: G36_m Cantidad Grabaciones: 545
Grabadora: G37_m Cantidad Grabaciones: 541
Grabadora: G40_m Cantidad Grabaciones: 555
Grabadora: G41_m Cantidad Grabaciones: 553
Grabado

In [8]:
df = pd.read_csv('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Complementary_Files/Datos.csv')

In [9]:
df_archivos = pd.DataFrame(columns=['Recorder', 'Date', 'Time', 'Path'])

# Ruta del directorio principal que contiene las carpetas y archivos
ruta_principal = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Jaguas_2018'

# Recorrer las carpetas y archivos dentro de la ruta principal
for raiz, carpetas, archivos in os.walk(ruta_principal):
    carpetas.sort()
    # Recorrer los archivos de cada carpeta
    for archivo in archivos:
        # Obtener la ruta completa del archivo
        ruta_archivo = os.path.join(raiz, archivo)
        
        # Obtener el nombre del archivo sin la extensión
        nombre_archivo, extension = os.path.splitext(archivo)
        
        # Separar el nombre del archivo en partes utilizando el caracter "_"
        partes_nombre = nombre_archivo.split("_")
        
        # Obtener los valores deseados de cada parte del nombre del archivo
        recorder = partes_nombre[0]
        fecha = partes_nombre[1]
        hora = partes_nombre[2]
        
        # Crear un diccionario con los valores del archivo actual
        datos_archivo = {
            'Recorder': recorder,
            'Date': fecha,
            'Time': hora,
            'Path': ruta_archivo
        }
        
        # Agregar el diccionario al dataframe
        df_archivos = df_archivos.append(datos_archivo, ignore_index=True)

# Convertir la columna 'Date' al formato 'yyyy/mm/dd'
df_archivos['Date'] = df_archivos['Date'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d').strftime('%Y/%m/%d'))

# Convertir la columna 'Time' al formato 'hh:mm:ss'
df_archivos['Time'] = df_archivos['Time'].apply(lambda x: pd.to_datetime(x, format='%H%M%S').strftime('%H:%M:%S'))

#Cargamos informacion adicional de las grabadoras
df = pd.read_csv('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Complementary_Files/Datos.csv')

#hacemos un Marge con el dataframe anteriormente creado, a base de recorrer los archivos
df_archivos = pd.merge(df_archivos, df, on="Recorder")

In [18]:
d2

{'g1': 12}

In [30]:
d = {'col1': ["g1", "g2", "g3", "g4", "g5"], 'col2': [3, 4, 5, 6, 7]}
d2 = {'col1': ["g4", "g1"], 'col2': [15, 20]}
df = pd.DataFrame(data=d)

label = 12
label_recorder = "g1"
d3 = {label_recorder: label}

df2 = pd.DataFrame(data=d2)
df =df.merge(df2, how="left")
df

,col1,col2
0,g1,3
1,g2,4
2,g3,5
3,g4,6
4,g5,7


In [31]:
model_name = "AE_batch_size_56_num_hiddens_64__day_17_hour_0_final.pth"
config = torch.load(f'Models/config_{model_name}')
model = AE(num_hiddens=config["num_hiddens"]).to(device)
dataset_test = torch.load(f'temporal/dataset_test_ae_jaguas_new.pth')
dataset_train = torch.load(f'temporal/dataset_train_ae_jaguas_new.pth')
model.load_state_dict(torch.load(f'Models/{model_name}', map_location=torch.device('cpu')))

y = torch.load("Features/training_labels_list.pth",  map_location=torch.device('cpu'))
X = torch.load("Features/training_samples_list_torch.pth",  map_location=torch.device('cpu'))
X = X[0:6000]
y["recorder"] = y["recorder"][0:6000]

scaler = StandardScaler().fit(X)
X_n = X
X_scaled = scaler.transform(X)
X_TSNE = TSNE(n_components=2, learning_rate="auto", init='random', random_state=0).fit_transform(X_scaled)

FileNotFoundError: ignored

In [ ]:
# Batch methods
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
import matplotlib.cm as cm

# Single methods
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from sklearn import metrics
from sklearn.metrics import silhouette_samples

In [ ]:
from logging import raiseExceptions
def plot_silhouette( X, cluster_labels, n_clusters, silhouette_avg, method, extra=""):
    fig, ax1 = plt.subplots(figsize=(12, 12))

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-1, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0,
                    0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
    print("Ya debio plotear")
    #plt.savefig(f"Clustering_Results/{method}/Figures/Silhouette_plot_{n_clusters}.pdf", format="pdf")
    #plt.show()

def plot_centroids(cluster_centers, testing, method, extra=""):
    plt.figure(figsize=(18, 18))
    testing._model.to("cpu")
    for i, spec in enumerate(cluster_centers):
        encodings = spec.reshape(64,9,9)
        encodings = torch.tensor(encodings).float()
        decodings = testing._model.decoder(encodings).detach().numpy()
        plt.subplot(6, 6, i + 1)
        plt.imshow(decodings[0, :, :], origin="lower", cmap="viridis")
        plt.xticks(())
        plt.yticks(())
    n_cluster = len(cluster_centers)
    #plt.savefig(f"Clustering_Results/{method}/Figures/Centroids_plot_{n_cluster}_{extra}.pdf", format="pdf")
    #plt.show()

import math

def num_rows_cols(num_elements):
    num_rows = int(math.sqrt(num_elements))
    num_cols = (num_elements + num_rows - 1) // num_rows
    return (num_rows, num_cols)

def get_row_col(pos, cols):
    row = pos // cols
    col = pos % cols
    return row, col

class Clustering_Results:
    def __init__(self, model, y, y_label="hour", hist_library="plt"):
        self._labels_cluster = None
        self._n_labels = None
        self._hist_library = hist_library
        self._label = y_label
        self._model = model
        self._n_clusters = len(set(model.labels_))
        self.y = y
        self._y = self.converter(y[self._label])
        self._n_labels = set(self._y)

    def converter(self, var):
        aux = []
        for i in range(len(var)):
            aux.append(var[i].item())
        return np.array(aux)

    def one_cluster_eval(self, cluster):
        index = np.where(self._model.labels_ == cluster)
        index = list(index[0])
        self._labels_cluster = self._y[index]
        return self._labels_cluster

    def joyplot(self):
        size_x = 8
        size_y = 6
        joy_vars = ["hour", "recorder"]
        for cluster in range(self._n_clusters):
            y_aux = []
            labels_cluster = []
            for i, label in enumerate(joy_vars):
                y_aux.append(self.converter(self.y[label]))
                index = np.where(self._model.labels_ == cluster)
                index = list(index[0])
                labels_cluster.append(y_aux[i][index])
            df = pd.DataFrame({'recorder':labels_cluster[0], "hour":labels_cluster[1]})
            joypy.joyplot(df, by="hour", column="recorder", range_style='own', 
                            grid="y", hist=False, linewidth=1, legend=False, figsize=(size_x,size_y),
                            title=f"Cluster {cluster} \nLabels distribution along recorders using recorders as rows",
                            colormap=cm.autumn_r, fade=False)
            joypy.joyplot(df, by="recorder", column="hour", range_style='own', 
                                grid="y", hist=False, linewidth=1, legend=False, figsize=(size_x,size_y),
                                title=f"Cluster {cluster} \nLabels distribution along recorders using hours as rows",
                                colormap=cm.autumn_r)
            plt.show()
            


    def histograms(self):
        bins = list(self._n_labels)
        num_rows, num_cols = num_rows_cols(self._n_clusters)
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(14, 14))
        if self._n_clusters <= 3:
                axes = np.expand_dims(axes,0)
                fig.set_figheight(6)
                fig.set_figwidth(12)
                if self._n_clusters == 1:
                    axes = np.expand_dims(axes,0)
                else:
                    pass
        else:
            pass
        for hist in range(self._n_clusters):
            aux = self.one_cluster_eval(hist)
            ax_0, ax_1 = get_row_col(hist, num_cols)
            if self._hist_library == "plt":
                axes[ax_0][ax_1].hist(aux, histtype="bar",
                                      color="paleturquoise", cumulative=False,
                                      edgecolor='black', 
                                      linewidth=1.2, bins=bins, stacked=False)
                axes[ax_0][ax_1].set_title(f"Cluster: {hist}", size=16)
            elif self._hist_library == "sns":
                sns.distplot(aux,bins=np.arange(aux.min(), aux.max()+1),
                             hist_kws=dict(edgecolor="black", linewidth=1), 
                             ax=axes[ax_0, ax_1])
                axes[ax_0][ax_1].set_title(f"Cluster: {hist}", size=16)              
            else:
                raise Exception(f"Library {self._hist_library} unused")
        plt.show()

In [ ]:
# from sklearn.cluster import KMeans
# Kmeans = KMeans(n_clusters=8, random_state=0).fit(X_scaled)
import warnings
warnings.filterwarnings("ignore")
Kmeans_Results = Clustering_Results(Kmeans, y, y_label="recorder", hist_library="plt")
Kmeans_Results.joyplot()